In [ ]:
# importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# EDA

In [ ]:
# importing the csv file

dataset = pd.read_csv(r'C:\Users\91976\Desktop\programming\AI and Ml\projects\survival predicton\Dataset.csv')

# size and shape of the dataset
print(dataset.shape)
print(f"size = {dataset.shape[0]*dataset.shape[1]}")

missing values

In [ ]:
features_with_nan = [features for features in dataset.columns if dataset[features].isnull().sum()>=1]

for features in features_with_nan:
    print(features , np.round(dataset[features].isnull().mean(),4) , '% missing values')

print(len(features_with_nan))    

numerical values

In [ ]:
numerical_features=[features for features in dataset.columns if dataset[features].dtype!='O']

print('Number of numerical features' , len(numerical_features))

In [ ]:
# discrete features

discrete_features = [features for features in numerical_features if len(dataset[features].unique())<=10 and features not in ['encounter_id','patient_id']]

print('discrete features', len(discrete_features))
print(discrete_features)


In [ ]:
# continous features

continous_features = [features for features in numerical_features if features not in discrete_features and features not in ['encounter_id','patient_id']] 

print("continous features" , len(continous_features))

In [ ]:
# lets analyse the continous values by creating histograms to understand the data 

for features in continous_features:
    data = dataset.copy()
    data[features].hist(bins=25)
    plt.xlabel(features)
    plt.ylabel("count")
    plt.title(features)
    plt.show()

categorical features

In [ ]:
categorical_features = [features for features in dataset.columns if dataset[features].dtype=='O']
print(len(categorical_features))

In [ ]:
for features in categorical_features:
    print(f"The feature is {features} and number of categories are {len(dataset[features].unique())}")

# Feature Engineering

missing values


In [ ]:
c_features_na = [features for features in dataset.columns if dataset[features].isnull().sum()>=1 and dataset[features].dtype=='O']

print(len(c_features_na))

In [ ]:
def replace_cat_features(dataset,c_features_na):
    data = dataset.copy()
    data[features] = data[features].fillna('Missing')
    return data


dataset = replace_cat_features(dataset,c_features_na)

dataset.head()

In [ ]:
n_features_na = [features for features in dataset.columns if dataset[features].isnull().sum()>=1 and dataset[features].dtype!='O']

for features in n_features_na:
    if(np.round(dataset[features].isnull().mean(),4) > 0.5 ):
        print(features)

In [ ]:
for features in n_features_na:
    median_values = dataset[features].median()
    dataset[features] = dataset[features].fillna(median_values)


print(dataset[n_features_na].isnull().sum())

# Feature selection


In [62]:
dataset.head()

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,...,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,66154,25312,118,0,68.0,22.730000,0,Caucasian,0.0,180.3,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular
1,114252,59342,81,0,77.0,27.420000,0,Caucasian,1.0,160.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory
2,119783,50777,118,0,25.0,31.950000,0,Caucasian,1.0,172.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic
3,79267,46918,118,0,81.0,22.640000,1,Caucasian,1.0,165.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular
4,92056,34377,33,0,19.0,27.654655,0,Caucasian,0.0,188.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trauma,Trauma


In [ ]:
corr = dataset.corr()[['hospital_death']].sort_values(by='hospital_death',ascending=False)

sns.heatmap(corr)
corr.head(7)

In [79]:
x = [ 'age' , 'weight' , 'bmi' , 'gender','d1_lactate_max' , 'd1_lactate_min' , 'apache_4a_hospital_death_prob','apache_4a_icu_death_prob' ]

dataset['gender'] = dataset['gender'].replace({'M':0 , 'F':1})

In [80]:
nan_x_features = [features for features in x if dataset[features].isnull().sum()>=1]
print(nan_x_features)
dataset['gender'].fillna(1)

['gender']


0        0.0
1        1.0
2        1.0
3        1.0
4        0.0
        ... 
91708    0.0
91709    1.0
91710    0.0
91711    1.0
91712    1.0
Name: gender, Length: 91713, dtype: float64

In [ ]:
dataset['gender'].fillna(1)

# Model building and selection 

In [81]:
# selecting the dependent and independent features
from sklearn.preprocessing import MinMaxScaler,StandardScaler

t_x = dataset[x]
t_y = dataset['hospital_death']


In [85]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y = train_test_split(t_x,t_y , train_size=0.95)

test_x.head()

,age,weight,bmi,gender,d1_lactate_max,d1_lactate_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob
30687,22.0,75.5,31.466187,1.0,1.9,1.5,0.05,0.02
29415,66.0,71.7,27.152663,0.0,1.9,1.5,0.05,0.02
31116,71.0,49.0,20.932120,1.0,1.9,1.5,0.17,0.10
41672,75.0,91.6,33.604801,1.0,5.0,2.0,0.07,0.04
55141,73.0,74.9,30.386628,1.0,1.9,1.5,0.01,0.00


In [86]:
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score


model = Pipeline([('scalar',MinMaxScaler()),
                    ('xgboost', XGBClassifier())])

model.fit(train_x,train_y) 
pred = model.predict(test_x)

print(pred)

acc = accuracy_score(pred,test_y)
print(acc)


[0 0 0 ... 0 0 0]
0.9219363279546445


In [83]:
import pickle

with open('Model.pkl','wb') as file:
    pickle.dump(model,file)